In [2]:
!pip show requests

Name: requests
Version: 2.25.1
Summary: Python HTTP for Humans.
Home-page: https://requests.readthedocs.io
Author: Kenneth Reitz
Author-email: me@kennethreitz.org
License: Apache 2.0
Location: c:\users\vega2\anaconda3\lib\site-packages
Requires: chardet, certifi, urllib3, idna
Required-by: Sphinx, jupyterlab-server, conda, conda-repo-cli, conda-build, anaconda-project, anaconda-client


In [3]:
!pip show beautifulsoup4

Name: beautifulsoup4
Version: 4.9.3
Summary: Screen-scraping library
Home-page: http://www.crummy.com/software/BeautifulSoup/bs4/
Author: Leonard Richardson
Author-email: leonardr@segfault.org
License: MIT
Location: c:\users\vega2\anaconda3\lib\site-packages
Requires: soupsieve
Required-by: conda-build


### 네이버 뉴스 제목 가져오기
* user-agent 헤더 값을 반드시 설정

In [26]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin


In [10]:
url = 'https://news.naver.com/'
req_header_dict = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.69 Safari/537.36'
}
res = requests.get(url, headers=req_header_dict)

print(res.status_code)
print(res.ok)

In [27]:
if res.ok:
    # 소스보기 했을 때 보여지는 텍스트
    html = res.text
    #BeautifulSoup 객체생성
    soup = BeautifulSoup(html, 'html.parser')
    #print(len(soup.select("a[href*='read.naver']")))
    atag_list = soup.select("a[href*='read.naver']")
    print(type(atag_list))
    for idx, atag in enumerate(atag_list,1):
        #print(type(atag))
        if atag:
            title = atag.text.strip()
            href = urljoin(url, atag['href'])
        if title:
            print(f'{idx} - {title} - {href}')

<class 'bs4.element.ResultSet'>
1 - ‘도이치모터스 주가 조작’ 권오수, 영장실질심사… - https://news.naver.com/main/read.naver?mode=LSD&mid=shm&sid1=102&oid=023&aid=0003653362
2 - “산업용 요소수 차량용 전환 여부, 추가 실험 진… - https://news.naver.com/main/read.naver?mode=LSD&mid=shm&sid1=105&oid=056&aid=0011157630
3 - 민주당 "초과세수 50조 넘는데 직무유기...홍남기 사과하라" - https://news.naver.com/main/read.naver?mode=LSD&mid=shm&sid1=100&oid=437&aid=0000280936
4 - 이철희 “文대통령 퇴임 때 ‘문전박대’ 소망”…무슨 의미? - https://news.naver.com/main/read.naver?mode=LSD&mid=shm&sid1=100&oid=020&aid=0003393893
5 - 러시아, ‘요격 시험’이라며 자국 인공위성 ‘미사일’로 부숴 - https://news.naver.com/main/read.naver?mode=LSD&mid=shm&sid1=101&oid=011&aid=0003985723
6 - 경기남부경찰, 수능날 105곳서 '수험생 태워주기' 제공 - https://news.naver.com/main/read.naver?mode=LSD&mid=shm&sid1=102&oid=001&aid=0012795499
7 - 고통받는 M세대...집값 폭등에 30대 집주인만 줄었다 - https://news.naver.com/main/read.naver?mode=LSD&mid=shm&sid1=101&oid=008&aid=0004671083
9 - 與, 윤석열 겨냥 “대장동 숨은 몸통 비호” 공세 - https://news.naver.com/main/read.naver?mode=LSD&mid=sh

In [ ]:
### 웹툰 이미지 다운로드 & 업로드
* referer 헤더 설정이 필요합니다.

In [ ]:
import requests
import os

In [ ]:
req_header = {
    'referer':'https://comic.naver.com/webtoon/detail?titleId=780506&no=32&amp;weekday=tue'
}
img_url_list = ['https://image-comic.pstatic.net/webtoon/780506/32/20211115143459_7181f6c9b607b12cf6f4da17353e77a3_IMAG01_1.jpg',
                'https://image-comic.pstatic.net/webtoon/780506/32/20211115143459_7181f6c9b607b12cf6f4da17353e77a3_IMAG01_2.jpg',
                'https://image-comic.pstatic.net/webtoon/780506/32/20211115143459_7181f6c9b607b12cf6f4da17353e77a3_IMAG01_3.jpg']

for img_url in img_url_list:
    res = requests.get(img_url, headers=req_header)
    print(res.status_code)
    if res.ok:
        #response 객체에 담겨진 binary data를 추출할때는 content 속성사용
        img_data = res.content
        file_name = os.path.basename(img_url)
        
        with open(file_name, 'wb') as file:
            print(f'writing to {file_name} ({len(img_data)}) bytes')
            file.write(img_data)

In [ ]:
### 이미지 업로드
* http://httpbin.org/post  URL로 다운받은 img 파일을 업로드
* requests의 post() 함수를 사용하고, files 속성에 image data를 지정한다

In [ ]:
import requests

upload_files_dict = {
    'img1': open('f1.jpg','rb'),
    'img2': open('f2.jpg','rb'),
    'img3': open('f3.jpg','rb')
}
url = "http://httpbin.org/post"
res = requests.post(url, files=upload_files_dict)
print(res.status_code)

In [ ]:
img1_data = res.json()['files']['img1']
#img1_data

In [ ]:
### 파파고 오픈 API 사용
* urllib 내부 라이브러리를 사용한 코드

In [ ]:
import os
import sys
import urllib.request

client_id = "Txlmbz8_3MnvRcDWLoZo" # 개발자센터에서 발급받은 Client ID 값
client_secret = "pmYRthuGCZ" # 개발자센터에서 발급받은 Client Secret 값

encText = urllib.parse.quote("Suddenly, I'm not half the man I used to be")
data = "source=en&target=ko&text=" + encText
url = "https://openapi.naver.com/v1/papago/n2mt"

request = urllib.request.Request(url)
request.add_header("X-Naver-Client-Id",client_id)
request.add_header("X-Naver-Client-Secret",client_secret)
response = urllib.request.urlopen(request, data=data.encode("utf-8"))
rescode = response.getcode()
if(rescode==200):
    response_body = response.read()
    print(response_body.decode('utf-8'))
else:
    print("Error Code:" + rescode)

In [ ]:
* requests를 사용한 코드로 변경하기

In [ ]:
import requests

client_id = "Txlmbz8_3MnvRcDWLoZo" # 개발자센터에서 발급받은 Client ID 값
client_secret = "pmYRthuGCZ" # 개발자센터에서 발급받은 Client Secret 값
url = "https://openapi.naver.com/v1/papago/n2mt"

#request parameter(요청 파라미터)
req_param_dict = {
    "source":"en",
    "target":"ko",
    "text":"Suddenly, I'm not half the man I used to be"
}
#request header(요청 헤더)
req_header_dict = {
    "X-Naver-Client-Id":client_id,
    "X-Naver-Client-Secret":client_secret
}

res = requests.post(url, data=req_param_dict, headers=req_header_dict)
print(res.ok)
print(f' 응답 헤더 : {res.headers}')
print(f' 요청 헤더 : {res.request.headers}')
print(f' 응답 코드 : {res.status_code}')  

In [ ]:
if res.ok:
    print(type(res.text))
    print(type(res.json()))
    print(res.text)
    print(res.json()['message']['result']['translatedText'])